# Tree Methods Consulting Project 

You've been hired by a dog food company to try to predict why some batches of their dog food are spoiling much quicker than intended! Unfortunately this Dog Food company hasn't upgraded to the latest machinery, meaning that the amounts of the five preservative chemicals they are using can vary a lot, but which is the chemical that has the strongest effect? The dog food company first mixes up a batch of preservative that contains 4 different preservative chemicals (A,B,C,D) and then is completed with a "filler" chemical. The food scientists beelive one of the A,B,C, or D preservatives is causing the problem, but need your help to figure out which one!
Use Machine Learning with RF to find out which parameter had the most predicitive power, thus finding out which chemical causes the early spoiling! So create a model and then find out how you can decide which chemical is the problem!

* Pres_A : Percentage of preservative A in the mix
* Pres_B : Percentage of preservative B in the mix
* Pres_C : Percentage of preservative C in the mix
* Pres_D : Percentage of preservative D in the mix
* Spoiled: Label indicating whether or not the dog food batch was spoiled.
___

**Think carefully about what this problem is really asking you to solve. While we will use Machine Learning to solve this, it won't be with your typical train/test split workflow. If this confuses you, skip ahead to the solution code along walk-through!**
____

# Good Luck!

In [15]:
#Tree methods Example
import findspark
findspark.init()
from pyspark.sql import SparkSession

from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import RandomForestClassifier,DecisionTreeClassifier

spark = SparkSession.builder.appName('dog').getOrCreate()

In [3]:
df = spark.read.csv('dog_food.csv',inferSchema=True,header=True)

In [4]:
df.printSchema()

root
 |-- A: integer (nullable = true)
 |-- B: integer (nullable = true)
 |-- C: double (nullable = true)
 |-- D: integer (nullable = true)
 |-- Spoiled: double (nullable = true)



This is not an appropriate question for ML RF but...

In [6]:
df.groupBy("Spoiled").mean().show()

+-------+-----------------+-----------------+------------------+------------------+------------+
|Spoiled|           avg(A)|           avg(B)|            avg(C)|            avg(D)|avg(Spoiled)|
+-------+-----------------+-----------------+------------------+------------------+------------+
|    0.0|5.422857142857143|             5.66|  8.01142857142857|5.6085714285714285|         0.0|
|    1.0|5.814285714285714|5.114285714285714|11.914285714285715| 5.507142857142857|         1.0|
+-------+-----------------+-----------------+------------------+------------------+------------+



In [32]:
assembler = VectorAssembler(inputCols=['D', 'B', 'A', 'C'],outputCol="features")
dfx = assembler.transform(df).select('features','Spoiled')
dfx.printSchema()

root
 |-- features: vector (nullable = true)
 |-- Spoiled: double (nullable = true)



In [33]:
dfx.head()

Row(features=DenseVector([3.0, 2.0, 4.0, 12.0]), Spoiled=1.0)

In [34]:
dt = DecisionTreeClassifier(labelCol='Spoiled',featuresCol='features')
dt_model = dt.fit(dfx)
dt_model.featureImportances

SparseVector(4, {0: 0.0149, 1: 0.0019, 3: 0.9832})

In [35]:
rf = RandomForestClassifier(labelCol='Spoiled',featuresCol='features')
rf_model = dt.fit(dfx)
rf_model.featureImportances

SparseVector(4, {0: 0.0149, 1: 0.0019, 3: 0.9832})

In [36]:
assembler = VectorAssembler(inputCols=['D', 'B', 'A'],outputCol="features")
dfx = assembler.transform(df).select('features','Spoiled')
dfx.printSchema()

root
 |-- features: vector (nullable = true)
 |-- Spoiled: double (nullable = true)



In [37]:
rf = RandomForestClassifier(labelCol='Spoiled',featuresCol='features')
rf_model = dt.fit(dfx)
rf_model.featureImportances

SparseVector(3, {0: 0.2624, 1: 0.3429, 2: 0.3947})

In [38]:
dt = DecisionTreeClassifier(labelCol='Spoiled',featuresCol='features')
dt_model = dt.fit(dfx)
dt_model.featureImportances

SparseVector(3, {0: 0.2624, 1: 0.3429, 2: 0.3947})